# 에볼라 강동 2011-10-01 부터 시작

In [1]:
# 열린 크롬창 내에 csv 버튼까지 보이게끔 해둬야 코드가 실행됨.

# 밑에 다운로드 한 이력이 있으면 안돌아감.

# 화면 크기 100%에서 줄이지 말기...

# 크롤링을 하려면 selenium, chromedriver, urllib이 필수적으로 필요하다.

chromedriver의 경우 Only local connections are allowed. 라는 이슈가 있기에, 서버에서는 사용 불가능하다. <br>
그리고 일반 크롬과 버전이 맞아야 사용 가능하므로, 사용전에 버전 확인을 하는 것이 좋다. <br>
따로 버전 확인을 하지 않더라도 에러가 다 알려주니 크게 신경쓰지 않아도 됨. <br>

[chromedriver.exe 다운받는 경로](https://chromedriver.chromium.org/downloads) <br>

(현재 주피터를 실행하는 경로에) 현재 내 pc의 chrome과 버전이 동일한 chromedriver.exe가 깔려만 있으면 잘 실행됨.

In [1]:
import sys, os
import requests #(네이버) api 이용할 때 쓰는 패키지
import selenium #크롤링 패키지

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
#

from pandas import DataFrame
from bs4 import BeautifulSoup #크롤링 패키지
import re #정규식 표현 모듈
from datetime import datetime
import pickle, progressbar, json, glob, time #pickle: 객체를 파일로 저장해주는 모듈, progressbar: 진행표시 바, json: 네이버 api, glob: 파일들 리스트 뽑을 때 사용
# https://wikidocs.net/83
from tqdm import tqdm #진행표시바
#

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import numpy as np

import urllib.request #파싱을 위한 모듈

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 크롬창 열기

In [2]:
print('브라우저를 실행시킵니다(자동 제어)\n')
browser = webdriver.Chrome('chromedriver') #크롬창열기 # (현재 주피터를 실행하는 경로에) 현재 내 크롬 버전과 동일한 chromedriver.exe가 있으면 실행됨.
# https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

news_url = 'https://www.kdca.go.kr/npt/biz/npp/ist/bass/bassSexdstnAgeStatsMain.do'

browser.get(news_url) #url 오픈
time.sleep(2)

# version issue
# SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 87
# Current browser version is 103.0.5060.114 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe

# solution
# https://codechacha.com/ko/selenium-chromedriver-version-error/

브라우저를 실행시킵니다(자동 제어)



# labeling을 위한 작업

## 시도lst

In [3]:
시도lst = ['서울', '부산', '대구', '인천',
      '광주', '대전', '울산', '경기', '강원', 
      '충북', '충남', '전북', '전남', '경북',
      '경남', '제주', '세종']
len(시도lst) # 17

17

## name2병id (len 63)

In [4]:
병id_lst = list(range(3, 19)) + list(range(20, 42)) + list(range(43, 68))
len(병id_lst)

63

In [5]:
급1병_lst = ['에볼라바이러스병', '마버그열', '라싸열', '크리미안콩고출혈열', '남아메리카출혈열', '리프트밸리열', '두창', '페스트',
            '탄저', '보툴리눔독소증', '야토병', '중증급성호흡기증후군(SARS)', '중동호흡기증후군(MERS)', '동물인플루엔자 인체감염증',
            '신종인플루엔자', '디프테리아']

급2병_lst = ['수두', '홍역', '콜레라', '장티푸스', '파라티푸스', '세균성이질', '장출혈성대장균감염증', 'A형간염', '백일해',
            '유행성이하선염', '풍진(2018년이전)', '풍진(선천성)', '풍진(후천성)', '폴리오', '수막구균 감염증',
            'b형헤모필루스인플루엔자', '폐렴구균 감염증', '한센병', '성홍열', '반코마이신내성황색포도알균(VRSA) 감염증',
            '카바페넴내성장내세균속균종(CRE) 감염증', 'E형간염']

급3병_lst = ['파상풍', 'B형간염', '일본뇌염', 'C형간염', '말라리아', '레지오넬라증', '비브리오패혈증', '발진티푸스', '발진열',
             '쯔쯔가무시증', '렙토스피라증', '브루셀라증', '공수병', '신증후군출혈열',
             '크로이츠펠트-야콥병(CDJ) 및 변종크로이츠펠트-야콥병(vCJD)', '황열', '뎅기열', '큐열', '웨스트나일열', '라임병',
             '진드기매개뇌염', '유비저', '치쿤구니야열', '중증열성혈소판감소증후군(SFTS)', '지카바이러스감염증']

병name_lst = 급1병_lst + 급2병_lst + 급3병_lst
len(병name_lst)

63

In [6]:
name2병id = {병name_lst[i] : 병id_lst[i] for i in range(len(병id_lst))}
len(name2병id)

63

## 지역 (len 272)

In [7]:
군구서울 = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구',
           '종로구', '중구', '중랑구']

군구부산 = ['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구']

군구대구 = ['남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구']

군구인천 = ['강화군', '계양구', '미추홀구', '남동구', '동구', '부평구', '서구', '연수구', '옹진군', '중구']

군구광주 = ['광산구', '남구', '동구', '북구', '서구']

군구대전 = ['대덕구', '동구', '서구', '유성구', '중구']

군구울산 = ['남구', '동구', '북구', '울주군', '중구']

군구경기 = ['가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시 소사구', '부천시 오정구', '부천시 원미구', '성남시 분당구', '성남시 수정구',
           '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구', '시흥시', '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시 기흥구', '용인시 수지구',
           '용인시 처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '양주군', '고양시 일산구', '포천군', '용인시', '남양주', '고양시 일산동', '안산시', '부천시']

군구강원 = ['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시', '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군']

군구충북 = ['괴산군', '단양군', '보은군', '영동군', '옥천군', '음성군', '제천시',' 증평군', '진천군', '청원군', '(구)청주시 상당구', '(구)청주시 흥덕구', '충주시', '청주시 상당구', '청주시 흥덕구', '청주시 서원구', '청주시 청원구']

군구충남 = ['계룡시', '공주시', '금산군', '논산시', '당진시', '보령시', '부여군', '서산시', '서천군', '아산시', '연기군', '예산군', '천안시', '청양군', '태안군', '홍성군', '천안시 동남구', '천안시 서북구']

군구전북 = ['고창군', '군산시' ,'김제시', '남원시', '무주군', '부안군', '순창군', '완주군', '익산시', '임실군', '장수군', '전주시 덕진구', '전주시 완산구', '정읍시', '진안군', '전주시']

군구전남 = ['강진군', '고흥군', '곡성군', '광양시', '구례군', '나주시', '담양군', '목포시', '무안군', '보성군', '순천시', '신안군', '여수시', '영광군', '영암군', '완도군', '장성군', '장흥군', '진도군', '함평군', '해남군', '화순군']

군구경북 = ['경산시', '경주시', '고령군', '구미시', '군위군', '김천시', '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시', '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군',
           '포항시 남구', '포항시 북구', '포항시']

군구경남 = ['거제시', '거창군', '고성군', '김해시', '남해군', '마산시', '밀양시', '사천시', '산청군', '양산시', '의령군', '진주시', '진해시', '창녕군', '창원시', '통영시', '하동군', '함안군', '함양군', '합천군', '창원시 마산합포구', '창원시 마산회원구',
           '창원시 성산구', '창원시 의창구', '창원시 진해구']

군구제주 = ['서귀포시', '제주시', '남제주군', '북제주군']

군구세종 = ['세종시']

### 각 시도별 군구 길이

In [8]:
for i in 시도lst:
    i, len(globals()[f'군구{i}'])

('서울', 25)

('부산', 16)

('대구', 8)

('인천', 10)

('광주', 5)

...

('전남', 22)

('경북', 25)

('경남', 25)

('제주', 4)

('세종', 1)

### 모든 시도의 모든 군구의 수

In [9]:
군구all = 군구서울 + 군구부산 + 군구대구 + 군구인천 + 군구광주 + 군구대전 + 군구울산 + 군구경기 + 군구강원 + 군구충북 + 군구충남 + 군구전북 + 군구전남 + 군구경북 + 군구경남 + 군구제주 + 군구세종
len(군구all)

272

# 날짜list 생성 및 초기화 함수

In [38]:
def 날짜list():
    # 윤년list (4년에 한 번씩)
    윤년list = [2020 - i*4 for i in range(5)]


    # 날짜list
    시작lst_ = []
    끝lst_ = []
    for 연도 in range(2001, 2022):
        for 월 in range(1, 13):
            if 월//10 < 1:
                월 = f'0{월}'
            시작lst_.append(f'{연도}-{월}-0{1}')

            if 월 in ['01', '03', '05', '07', '08', 10, 12]:
                끝lst_.append(f'{연도}-{월}-{31}')

            elif 월 in ['04', '06', '09', 11]:
                끝lst_.append(f'{연도}-{월}-{30}')

            elif 월 == '02':
                if 연도 not in 윤년list:
                    끝lst_.append(f'{연도}-{월}-{28}')
                else:
                    끝lst_.append(f'{연도}-{월}-{29}')



    ## 머리가 아프니, 2022년은 따로 만들어서 붙이도록 한다.
    for 월 in range(1, 7):
        if 월//10 < 1:
            월 = f'0{월}'
        시작lst_.append(f'{2022}-{월}-0{1}')
        if 월 in ['01', '03', '05', '07', '08', 10, 12]:
            끝lst_.append(f'{2022}-{월}-{31}')

        elif 월 in ['04', '06', '09', 11]:
            끝lst_.append(f'{2022}-{월}-{30}')

        elif 월 == '02':
            if 2022 not in 윤년list:
                끝lst_.append(f'{2022}-{월}-{28}')
            else:
                끝lst_.append(f'{2022}-{월}-{29}')
    return 시작lst_, 끝lst_

In [53]:
시작lst, 끝lst = 날짜list()
시작lst[:5]
끝lst[:5]

len(시작lst)
len(끝lst)
## 258개여야함. (12*21 + 6)

['2001-01-01', '2001-02-01', '2001-03-01', '2001-04-01', '2001-05-01']

['2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31']

258

258

# 옵션버튼 클릭

In [156]:
def delete(input_box):
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)

# 통계작성_다운로드, 매 loop마다 쓰이므로 함수로 작성해서 집어넣도록 하겠습니다.
def 통계작성_다운로드():
    ## 통계작성버튼 클릭
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/input[2]').click()
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/input[2]').click()
#     browser.implicitly_wait(2)
    time.sleep(5) # 여기 시간 늘린게 가장 큰 성공 요인인 것 같음.

    ## csv 다운로드 클릭 //*[@id="csvDownBtn"] //*[@id="csvDownBtn"]/img
    browser.find_element_by_xpath('//*[@id="csvDownBtn"]').click()
#     browser.implicitly_wait(3)
    time.sleep(5)

# 객체.implicitly_wait(시간)은 작동? 완료 되면 그냥 무시되는 코드




def 공통(병명_):
    ## 연령별 성별 탭 클릭
    search_opt_box = browser.find_element_by_xpath('//*[@id="ui-id-3"]')
    search_opt_box.click() #버튼 클릭
    browser.implicitly_wait(0.02)

#####------------------------------------------------------------------------------------------------------------------- 수정
    # 병 리셋
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/button/div').click()
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[1]/label/input').click() # 전체
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[1]/label/input').click() # 전체 (병 선택하기 전에 두 번 클릭해서 체크박스 리셋)
    browser.implicitly_wait(0.02)
    time.sleep(1)

    # 병 선택
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[{병명_}]/label/input').click() # 에볼라

    ## 끝에는 닫아주기까지 해야함.
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/button/div').click()
#     browser.find_element_by_xpath('//*[@id="tabs-3"]').click() # 빈 화면 클릭하기. tnex에서 실행시 필요, 집에선 끄기
#####-------------------------------------------------------------------------------------------------------------------
    
    # 연령 선택
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm_ageType2"]').click()
    

## 창 새로 열기 전에 하던 시도군구부터 다시 시작해야하므로
def 시도군구(시도_, 군구_):
    ## 시도
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]').click()
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]/option[{시도_+2}]').click()
    ## 군구
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구_+2}]').click()



    
# 기간 선택, 함수로 만들어서 한 줄로 실행하는 코드를 집어넣도록 하겠습니다.
def 기간선택(시도_, 군구_, 병명_):
    for idx in range(len(시작lst)):
        print(f'시작 날짜입니다. {시작lst[idx]}')
        
        # 8개월이 지나면 창 다시 실행.
        if idx != 0 and idx%12 == 0: # 이것도 에러 잡은 성공 요인.
#             다시실행()
            browser.refresh()
            time.sleep(1)
            공통(병명_)
            time.sleep(0.5)
            시도군구(시도_,군구_)
            

        ## 시작 날짜
        search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmStartDt"]')
        delete(search_box)
        browser.implicitly_wait(0.1)
        search_box.send_keys(시작lst[idx])
        search_box.send_keys(Keys.RETURN)
        browser.implicitly_wait(0.1)

        ## 끝 날짜
        search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmEndDt"]')
        delete(search_box)
        browser.implicitly_wait(0.1)
        search_box.send_keys(끝lst[idx])
        search_box.send_keys(Keys.RETURN)
        browser.implicitly_wait(0.1)
        
#         ## 닫아주기
#         browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[1]/li[2]/img[2]').click()
        
#         browser.find_element_by_xpath('//*[@id="tabs-3"]').click() # 빈 화면 클릭하기. tnex에서 실행시 필요, 집에선 끄기
#         browser.find_element_by_xpath('//*[@id="tabs-3"]').click() # 빈 화면 클릭하기. tnex에서 실행시 필요, 집에선 끄기
        time.sleep(0.3)

        # 8개월되기 전(7개월째)의 정보 저장
        if idx%11 == 0:
            시도 = 시도_
            군구 = 군구_
        
        ## 닫아주기
#         browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[1]/li[2]/img[2]').click()
        
        통계작성_다운로드()

```
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[1]/label/input') # 전체 (병 선택하기 전에 두 번 클릭해서 체크박스 리셋)
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[2]/label/input') # 1급
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[3]/label/input').click() # 에볼라
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[4]/label/input') # 마버그
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[19]/label/input') # 2급
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[20]/label/input') # 수두
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[42]/label/input') # 3급
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[67]/label/input') # 마지막 병
```

# 다시 작성
어차피 자동화 안될거 그냥 하나하나 하기로...
### 구 싹 돌고난 다음에야 그 다음 병으로 넘어가는 식으로...

In [45]:
시도lst

['서울',
 '부산',
 '대구',
 '인천',
 '광주',
 '대전',
 '울산',
 '경기',
 '강원',
 '충북',
 '충남',
 '전북',
 '전남',
 '경북',
 '경남',
 '제주',
 '세종']

In [157]:
browser.refresh() # 새로고침

In [117]:
병name_lst[1]

'마버그열'

In [158]:
### 바꿔줘야 합니다. 잘 짜여졌습니다. 병 각각, 군구 각각
# browser.find_element_by_xpath('//*[@id="tabs-3"]').click() # 빈 화면 클릭하기. tnex에서 실행시 필요, 집에선 끄기
병명 = name2병id['에볼라바이러스병'] # 에볼라 병
시도 = 시도lst.index('서울') # 서울
군구 = 1
print(군구서울[군구])





공통(병명)


## 시도
print(f'{시도lst[시도]}를(을) 돌리고 있습니다.')
browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]').click()
browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]/option[{시도+2}]').click()

## 군구
print(f'{시도lst[시도]}의 군구 {군구}를(을) 돌리고 있습니다.', datetime.today().time().strftime('%H:%M:%S'))
browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
기간선택(시도, 군구, 병명)

강동구
서울를(을) 돌리고 있습니다.
서울의 군구 1를(을) 돌리고 있습니다. 00:16:40
시작 날짜입니다. 2009-02-01
시작 날짜입니다. 2009-03-01
시작 날짜입니다. 2009-04-01
시작 날짜입니다. 2009-05-01
시작 날짜입니다. 2009-06-01
시작 날짜입니다. 2009-07-01
시작 날짜입니다. 2009-08-01
시작 날짜입니다. 2009-09-01
시작 날짜입니다. 2009-10-01
시작 날짜입니다. 2009-11-01
시작 날짜입니다. 2009-12-01
시작 날짜입니다. 2010-01-01
시작 날짜입니다. 2010-02-01
시작 날짜입니다. 2010-03-01
시작 날짜입니다. 2010-04-01
시작 날짜입니다. 2010-05-01
시작 날짜입니다. 2010-06-01
시작 날짜입니다. 2010-07-01
시작 날짜입니다. 2010-08-01
시작 날짜입니다. 2010-09-01
시작 날짜입니다. 2010-10-01
시작 날짜입니다. 2010-11-01
시작 날짜입니다. 2010-12-01
시작 날짜입니다. 2011-01-01
시작 날짜입니다. 2011-02-01
시작 날짜입니다. 2011-03-01
시작 날짜입니다. 2011-04-01
시작 날짜입니다. 2011-05-01
시작 날짜입니다. 2011-06-01
시작 날짜입니다. 2011-07-01
시작 날짜입니다. 2011-08-01
시작 날짜입니다. 2011-09-01
시작 날짜입니다. 2011-10-01


ElementClickInterceptedException: Message: element click intercepted: Element <input type="button" value="통계작성" class="btn-select bassSexdstnAgeStatsBtn"> is not clickable at point (631, 402). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 5.11204e-05; cursor: default; position: fixed;"></div>
  (Session info: chrome=103.0.5060.114)


# 멈춘 부분 부터 다시 시작
(옵션버튼 함수 다시 실행 안해줘도 됨.)

## 날짜 멈춘 부분

In [159]:
멈춘부분 = 시작lst.index('2011-10-01')
시작lst = 시작lst[멈춘부분:]
끝lst = 끝lst[멈춘부분:]

In [160]:
시작lst[0]
시작lst[-1]

'2011-10-01'

'2022-06-01'

## 크롬창 다시 열 때 멀리 안가기 위해 가져왔습니다.

In [ ]:
print('브라우저를 실행시킵니다(자동 제어)\n')
browser = webdriver.Chrome('chromedriver') #크롬창열기

news_url = 'https://www.kdca.go.kr/npt/biz/npp/ist/bass/bassSexdstnAgeStatsMain.do'

browser.get(news_url) #url 오픈
time.sleep(2)

## 날짜 초기화

In [114]:
시작lst, 끝lst = 날짜list()
시작lst[:5]
끝lst[:5]

len(시작lst)
len(끝lst)
## 258개여야함. (12*21 + 6)

['2001-01-01', '2001-02-01', '2001-03-01', '2001-04-01', '2001-05-01']

['2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31']

258

258

In [5]:
len(시작lst[:시작lst.index('2013-03-01')]) # 내가 해냄,,,

146

In [6]:
len(시작lst)

258